In [1]:
import pandas as pd
import psycopg2

In [2]:
server = 'localhost'
database = 'python_etl'
user = 'postgres'
password = 'root'

connection_string = f"dbname={database} user={user} password={password} host={server}"

try:
    conn = psycopg2.connect(connection_string)
    print("Conexão bem-sucedida ao PostgreSQL!")
    
    cursor = conn.cursor()
except Exception as e:
    print("Erro ao conectar ao PostgreSQL:", e)

Conexão bem-sucedida ao PostgreSQL!


In [3]:
dados = pd.read_excel("/home/juniorsilvadev/engenharia-dados/python-etl/Arquivos Excel/arquivos_excel/Ordens.xlsx")

In [24]:
# Converter a coluna 'created_at' para data e hora
dados['created_at'] = pd.to_datetime(dados['created_at'])

In [25]:
str(dados.columns).replace("'", "")

'Index([id, created_at, customer_id, status], dtype=object)'

In [26]:
create_table_query = """
CREATE TABLE IF NOT EXISTS ordens (
    id SERIAL PRIMARY KEY,
    created_at TIMESTAMP,
    customer_id INTEGER,
    status VARCHAR(255)
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()
    print("Tabela 'ordens' criada com sucesso!")
except Exception as e:
    print("Erro ao criar a tabela 'categorias':", e)
    conn.rollback()

Tabela 'ordens' criada com sucesso!


In [27]:
cursor.execute("TRUNCATE TABLE ordens")

In [28]:
insert_query = """
INSERT INTO ordens (id, created_at, customer_id, status)
VALUES (%s, %s, %s, %s)
"""

try:
    if conn.closed:
        conn = psycopg2.connect(connection_string)
    
    cursor = conn.cursor()
    
    for index, row in dados.iterrows():
        cursor.execute(insert_query, (
            int(row['id']), 
            row['created_at'], 
            int(row['customer_id']), 
            str(row['status'])
        ))
    
    conn.commit()
    print("Dados inseridos com sucesso!")
except Exception as e:
    print("Erro ao inserir dados:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()

Dados inseridos com sucesso!
